In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from response.feature import Feature

In [3]:
import os

def json2data(path):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            max_ = 300
            for i, data in enumerate( json_data[mode] ) :
                if i > max_:
                    break
                text = data["data"]
                label = data["label"][0]
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [4]:
label_list = "YN WH please proposal plain".split()
label_dict = dict( zip(label_list, range(len(label_list))) )

def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        y.append(label_dict[la])
    return X, y

In [5]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../../corpus/question/'

In [6]:
df = json2data(data_path)
df

,text,label,subLabel
0,メニューを見せていただけますか？,please,
1,おいでいただけますか？,please,
2,マッシュポテトをもらえますか？,please,
3,伝言を預かっていただけますか？,please,
4,ご一緒しませんか？,please,
...,...,...,...
1236,メンバーになりたいかい？,YN,
1237,明日スタートなさいますか？,YN,
1238,トレイがいりますか？,YN,
1239,2階も見たいですか？,YN,


In [7]:
df["label"].value_counts()

proposal    301
plain       301
YN          291
please      241
WH          107
Name: label, dtype: int64

In [8]:
X, y = extract_X_y(df)

In [9]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [10]:
f = Feature()

In [11]:
please = df[df["label"]=="please" ]

In [12]:
please = list(please.text.values)

In [13]:
please

['メニューを見せていただけますか？',
 'おいでいただけますか？',
 'マッシュポテトをもらえますか？',
 '伝言を預かっていただけますか？',
 'ご一緒しませんか？',
 'お手伝いいただくことはできますか？',
 'もう少し詳しく説明していただけますか。',
 'もう一度言っていただけますか。',
 'もう一度チャンスをくれないかな？',
 '私がご一緒してもいいですか。',
 'いくつかお試しになりたいですか。',
 'どれかすすめていただけますか。',
 '遅れそうです。席をとっておいてくれますか？',
 'もう一度あの引用文を言ってくれる？',
 '少し小さな声で話してくれる？',
 'それはどんなものだったのか私に説明してもらえますか？',
 '彼女への電話はあとにしてもらえますか？',
 '私と席を変わってくれますか？',
 'エアコンを少し下げていただけますか？',
 'このカードに必要事項を記入してから、こちらにご署名いただけますか。',
 '車を工場へ入れてくれ。',
 'お父さんと話をさせてもらえませんか？',
 '今朝は駅まで乗せてもらえるかな？',
 '皆さんに自己紹介をお願いできますか？',
 '彼を起こしてくれるかしら？',
 'あなたに華郷旅館をお願いできるかしら？',
 '来週の火曜までに提案書を用意してもらえるかしら。',
 'どんな人だったか説明していただけますか。',
 '今日のあなたの営業訪問の予定を聞かせてくれるかしら。',
 '鉢植えの松の木を舞台上に運んでくれる？',
 'みんなが、ここに泊まりたくなるような写真をずっと撮り続けてくれるかい？',
 'デービッドにわたしたち家族全員から「ありがとう」と伝えてもらえないかしら？',
 '乾杯の音頭を取っていただけますか？',
 '図書館への行き方を教えていただけますか？',
 '桜山公園からゴールまでの道順を教えていただけませんか？',
 'あなたはもしかしてレプラコーンでいらっしゃいますか？',
 'わたしを彼に紹介してくれないかい？',
 'ジョン先生をお願いできる？',
 'グレープソーダとポテトチップスを買ってきてくれる？',
 'ぼくは、Mサイズのフライドポテトとアップルジュースをもらえますか？',
 '私のコンピューターをモニターにつないでくれません

In [14]:
F = Feature()
F.make_features(please)

見せ
いただけ


In [15]:
F.show_features()

features num : 6791
feature type : f_is_contain_independent, nums : 2
0 : not independet
1 : independet

feature type : f_contain_keyword, nums : 16
2 : なぜ
3 : どれくらい
4 : どこ
5 : 見せ-て-いただけ
6 : だれ
7 : どうですか
8 : どんな
9 : どう
10 : みたら
11 : の？
12 : どうか
13 : 誰
14 : どうやって
15 : うか？
16 : 何
17 : いつ

feature type : f_pos_order, nums : 1865
18 : 助詞-格助詞_動詞-非自立可能_助詞-接続助詞_動詞-非自立可能
19 : 助詞-格助詞_補助記号-読点_代名詞_助詞-格助詞
20 : 名詞-普通名詞-形状詞可能_助動詞_補助記号-句点
21 : 代名詞_助詞-格助詞_名詞-普通名詞-一般
22 : 動詞-一般_助動詞_名詞-普通名詞-副詞可能
23 : 助動詞_助動詞_補助記号-句点_EOS
24 : 動詞-非自立可能_動詞-非自立可能
25 : FOS_名詞-固有名詞-地名-一般_助詞-格助詞
26 : 助詞-接続助詞_形容詞-非自立可能_補助記号-句点_EOS
27 : 助詞-準体助詞_助動詞_助詞-接続助詞_補助記号-読点
28 : 名詞-普通名詞-一般_助詞-格助詞_動詞-一般_形状詞-助動詞語幹
29 : 動詞-非自立可能_名詞-普通名詞-副詞可能_助詞-格助詞
30 : 名詞-普通名詞-一般_名詞-普通名詞-一般_補助記号-一般
31 : 助詞-格助詞_代名詞_助詞-格助詞_助詞-副助詞
32 : 助詞-格助詞_動詞-一般_助詞-準体助詞_助詞-格助詞
33 : 名詞-普通名詞-一般_助詞-格助詞_連体詞_名詞-普通名詞-サ変可能
34 : 助詞-係助詞_代名詞_助詞-格助詞
35 : 補助記号-読点_副詞_名詞-数詞
36 : 動詞-非自立可能_名詞-普通名詞-一般
37 : 接尾辞-名詞的-一般_補助記号-読点_名詞-普通名詞-一般
38 : FOS_名詞-固有名詞-人名-一般_接尾辞-名詞的-一般_助詞-格助詞
39 : 助動詞_助動詞_補

In [16]:
t = "美術館への行き方を教えていただけますか？"
t = "私と席を変わってくれますか？"
# t = "次の週末、マンゴータウンに来ない？"
doc = nlp(t)

In [21]:
import ginza
for i, token in enumerate(doc):
    # print(token.orth_, token.pos_, ginza.inflection(token))
    if token.pos_=="VERB" or "AUX":
        inflection = ginza.inflection(token)
        if inflection=="":
            continue
        conj = ginza.inflection(token).split(",")[1].split("-")[0]
        if conj=="連用形" and i< (len(doc)-1):
            print(token)
            if doc[i+1].orth_=="て": 
                print(token, doc[i+1], min(3, len(doc)-1-i))
                f = "-".join( [token.orth_ for token in doc[i:i+min(3, len(doc)-1-i)] ] )
                print(f)   
    else:
        print("")

変わっ
変わっ て 3
変わっ-て-くれ
くれ
